In [1]:
import pandas as pd

name = 'bio01i'

csvFileName = "/home/peter/Projects/wifo/data/productspace/patent/{}-tree-base.csv".format(name)

df = pd.read_csv(csvFileName)
# Drop numbers
df.drop("Unnamed: 0", axis=1, inplace=True)
df.head()
print("Shape:", df.shape)

Shape: (5390165, 2)


In [2]:
#substrdf['ipc']
hierarchy_digits = [1, 3, 4, 8, '']
for d in hierarchy_digits[:-1]:
    df['ipc{}'.format(d)] = df.ipc.str[:d]
df.head()

,appln_id,ipc,ipc1,ipc3,ipc4,ipc8
0,12,B01J0021/10,B,B01,B01J,B01J0021
1,12,B01J0023/00,B,B01,B01J,B01J0023
2,12,B01J0023/08,B,B01,B01J,B01J0023
3,12,B01J0023/22,B,B01,B01J,B01J0023
4,12,B01J0023/28,B,B01,B01J,B01J0023


In [3]:
from tree_partitioning import *

root = Node("root")

def add_nodes(node, data, step = 0):
    if step == len(hierarchy_digits):
        node.weight = data.shape[0]
        return 
    col_name = 'ipc{}'.format(hierarchy_digits[step])
    for v in data[col_name].unique():
        if step < 1:
            print("-" * step + v)
        t = Node(v)
        node.add_child(t)
        add_nodes(t, data[data[col_name] == v], step = step + 1)

add_nodes(root, df)

B
C
G
A
D
H
E
F


In [4]:
weight = root.calc_subtree_weights()
print("Weight:", weight)
if df.shape[0] != weight:
    raise Exception("Double counts!")

Weight: 5390165.0


In [17]:
target_weight = 10000

e, p = root.find_partition(target_weight=target_weight)
print("Error:", e)
print("Partition Count:", len(p))
#print("\n".join(str(subtree) for subtree in p))
#print("\n".join(subtree.name, subtree.weight for subtree in p))


Error: 219401399403.0
Partition Count: 1493


In [18]:
print(root.children[0])
print(root.children[1])
print(root.children[2])

print(root.children[1].children[1])
print(root.children[1].children[1].children[0])
print(root.children[1].children[1].children[0].children[6])

[root -> 0] Name: B, Weight: 83905.0
[root -> 1] Name: C, Weight: 2664690.0
[root -> 2] Name: G, Weight: 614746.0
[root -> 1 -> 1] Name: C12, Weight: 1818200.0
[root -> 1 -> 1 -> 0] Name: C12N, Weight: 1087027.0
[root -> 1 -> 1 -> 0 -> 6] Name: C12N0003, Weight: 401.0


In [19]:
import csv
with open('classification-{}-{}.csv'.format(target_weight, name), 'w') as f:
    writer = csv.writer(f)
    for subtree in p:
        writer.writerow([subtree.name, subtree.weight])